Test Data 1 : KuangDB, Test Data 2 : FIDSDB

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import matplotlib.pyplot as plt
import cv2
import glob
import os
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.models import load_model

In [2]:
def load_dataset(folderdataset,stat):
    image = []
    label = []

    if stat==1:
        for f in os.listdir(os.path.dirname(folderdataset)):
            if f!='.DS_Store':
                path = folderdataset+f
                l = findLabel(f)
                for filename in os.listdir(path):
                    if filename !='.DS_Store':
                        file = path+'/'+filename
                        img = np.asarray(cv2.imread(file))
                        img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
                        image.append(img)
                        if l=='apple':
                            lab=1
                        elif l=='banana':
                            lab=2
                        elif l=='lemon':
                            lab=3
                        elif l=='lime':
                            lab=4
                        elif l=='orange':
                            lab=5
                        elif l=='peach':
                            lab=6
                        elif l=='pear':
                            lab=7
                        label.append(lab)
    elif stat==2:
        for filename in os.listdir(os.path.dirname(folderdataset)):
            if filename!='.DS_Store':
                file = folderdataset+filename
                img = np.asarray(cv2.imread(file))
                img = cv2.resize(img, (128, 128), interpolation = cv2.INTER_AREA)
                image.append(img)
                l = findLabel(filename)
                if l=='apple':
                    lab=1
                elif l=='banana':
                    lab=2
                elif l=='lemon':
                    lab=3
                elif l=='lime':
                    lab=4
                elif l=='orange':
                    lab=5
                elif l=='peach':
                    lab=6
                elif l=='pear':
                    lab=7
                label.append(lab)
    
    return image,label

In [3]:
def findLabel(text):
    label = ""
    for i in range(0,len(text)):
        if text[i] != '_':
            label+= text[i]
        elif text[i]=='_':
            break
    return label

In [4]:
X_train,y_train = load_dataset("trainData_RGBD-object/",1)
X_test1,y_test1 = load_dataset("testData-1_KuangDB/",1)
X_test2,y_test2 = load_dataset("testData-2_FIDSDB/",2)

In [5]:
print("Number of Train Data : ",np.shape(X_train)[0])
print("Number of Test Data 1 : ",np.shape(X_test1)[0])
print("Number of Test Data 2 : ",np.shape(X_test2)[0])

Number of Train Data :  21287
Number of Test Data 1 :  300
Number of Test Data 2 :  232


In [7]:
X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_test1 = np.asarray(X_test1)
y_test1 = np.asarray(y_test1)
X_test2 = np.asarray(X_test2)
y_test2 = np.asarray(y_test2)

In [8]:
model = keras.Sequential([
    keras.layers.Conv2D(16, (5,5), activation='relu', input_shape=(128,128,3)),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(32, (5,5), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(64, (5,5), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Conv2D(128, (5,5), activation='relu'),
    keras.layers.MaxPooling2D(),
    keras.layers.Flatten(),
    keras.layers.Dense(1024, activation='relu'),
    keras.layers.Dense(256, activation='relu'),
    keras.layers.Dense(8, activation='softmax')
])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 124, 124, 16)      1216      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 62, 62, 16)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 58, 58, 32)        12832     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 29, 29, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 25, 25, 64)        51264     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 12, 12, 64)        0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 8, 8, 128)         2

In [9]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [10]:
model.fit(X_train, y_train, epochs=25, shuffle=True, validation_split=0.2)

Train on 17029 samples, validate on 4258 samples
Epoch 1/25
17029/17029 [==============================] - 291s 17ms/sample - loss: 0.2934 - accuracy: 0.9245 - val_loss: 0.2925 - val_accuracy: 0.9049
Epoch 2/25
17029/17029 [==============================] - 259s 15ms/sample - loss: 0.0621 - accuracy: 0.9821 - val_loss: 0.3383 - val_accuracy: 0.9317
Epoch 3/25
17029/17029 [==============================] - 257s 15ms/sample - loss: 0.1013 - accuracy: 0.9715 - val_loss: 0.7518 - val_accuracy: 0.8671
Epoch 4/25
17029/17029 [==============================] - 258s 15ms/sample - loss: 0.0346 - accuracy: 0.9901 - val_loss: 0.3132 - val_accuracy: 0.9338
Epoch 5/25
17029/17029 [==============================] - 260s 15ms/sample - loss: 0.0220 - accuracy: 0.9941 - val_loss: 0.4637 - val_accuracy: 0.9310
Epoch 6/25
17029/17029 [==============================] - 258s 15ms/sample - loss: 1.1645e-04 - accuracy: 1.0000 - val_loss: 0.4739 - val_accuracy: 0.9265
Epoch 7/25
17029/17029 [=================

In [11]:
loss, accuracy = model.evaluate(X_train, y_train, verbose=0)
print('Train Loss :', loss)
print('Train Accuracy :', accuracy)

Train Loss : 0.225307653119779
Train Accuracy : 0.98130316


In [14]:
result1 = model.predict_classes(tf.cast(X_test1, tf.float32)) 
print("Test Data 1 Accuracy : ", accuracy_score(y_test1, result1) * 100)

Test Data 1 Accuracy :  20.0


In [15]:
print(classification_report(y_test1, result1))

              precision    recall  f1-score   support

           1       0.00      0.00      0.00        50
           2       0.09      0.10      0.10        50
           3       0.62      0.16      0.25        50
           4       0.00      0.00      0.00         0
           5       0.31      0.08      0.13        50
           6       0.50      0.18      0.26        50
           7       0.19      0.68      0.30        50

    accuracy                           0.20       300
   macro avg       0.24      0.17      0.15       300
weighted avg       0.28      0.20      0.17       300



/Users/faoezanf/opt/anaconda3/envs/env1/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [17]:
result2 = model.predict_classes(tf.cast(X_test2, tf.float32)) 
print("Test Data 2 Accuracy : ", accuracy_score(y_test2, result2) * 100)

Test Data 2 Accuracy :  19.396551724137932


In [18]:
print(classification_report(y_test2, result2))

              precision    recall  f1-score   support

           1       0.12      0.13      0.12        38
           2       0.30      0.24      0.27        42
           3       0.50      0.10      0.17        29
           4       1.00      0.14      0.24        29
           5       0.50      0.06      0.10        35
           6       0.00      0.00      0.00        27
           7       0.15      0.66      0.25        32

    accuracy                           0.19       232
   macro avg       0.37      0.19      0.16       232
weighted avg       0.36      0.19      0.17       232



In [19]:
print("Confusion Matriks Data Test 1:")
print(confusion_matrix(y_test1, result1))

Confusion Matriks Data Test 1:
[[ 0 13  0  0  3  6 28]
 [ 1  5  3  2  1  0 38]
 [ 1  2  8  5  2  0 32]
 [ 0  0  0  0  0  0  0]
 [ 5  1  2  1  4  2 35]
 [ 0 25  0  1  2  9 13]
 [ 0  7  0  7  1  1 34]]


In [20]:
print("Confusion Matriks Data Test 2:")
print(confusion_matrix(y_test2, result2))

Confusion Matriks Data Test 2:
[[ 5  6  0  0  1  3 23]
 [ 5 10  3  0  1  0 23]
 [ 6  0  3  0  0  0 20]
 [ 5  5  0  4  0  1 14]
 [13  0  0  0  2  0 20]
 [ 4  6  0  0  0  0 17]
 [ 5  6  0  0  0  0 21]]
